In [1]:
import tensorflow.compat.v1 as tf  
tf.disable_v2_behavior()   # tensorflow v2 使用 v1 的语句

import os
import pickle
import numpy as np

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto(allow_soft_placement = True)
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.6)
config.gpu_options.allow_growth = True

CIFAR_DIR = "./cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

Instructions for updating:
non-resource variables are not supported in the long term
['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [2]:
def load_data(filename):
    """read data from data file"""
    with open(filename, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']
    
class CifarData:
    def __init__(self, filenames, need_shuffle):
        #训练集需要shuffle，测试集不需要shuffle
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 - 1
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0 #当前到数据集哪个位置
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        # [0,1,2,3,4,5] -> [5,3,2,4,0,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    def next_batch(self, batch_size):
        """return batch_size examples as a batch"""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [4]:
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])

hidden1 = tf.layers.dense(x, 100, activation=tf.nn.relu)
hidden2 = tf.layers.dense(hidden1, 100, activation=tf.nn.relu)
hidden3 = tf.layers.dense(hidden2, 50, activation=tf.nn.relu)

y_ = tf.layers.dense(hidden3, 10)


#两类时使用的sigmoid损失函数

"""
p_y_1 = tf.nn.sigmoid(y_)

y_reshape = tf.reshape(y, (-1, 1))
y_reshape_float = tf.cast(y_reshape, tf.float32)
loss = tf.reduce_mean(tf.square(y_reshape_float - p_y_1))

predict = p_y > 0.5
correct_prediction = tf.equal(tf.cast(predict, tf.int64), y_reshape)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))
"""
"""
#多类时使用的softmax损失函数 + mean square loss
p_y = tf.nn.softmax(y_)
# 5 -> [0, 0,0,0,1,0,0,0,0,0]
y_one_hot = tf.one_hot(y, 10, dtype=tf.float32)
y_reshape = tf.reshape(y, (-1, 1))
y_reshape_float = tf.cast(y_reshape, tf.float32)
loss = tf.reduce_mean(tf.square(y_one_hot - p_y))
"""

#交叉熵损失函数
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# y_ -> softmax
# y -> one_hot
# loss = ylogy_

# indices
predict = tf.argmax(y_, 1)
# 
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('trian_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [5]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 50000
test_steps = 100

with tf.Session() as sess:
    sess = tf.InteractiveSession(config = config)
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op], #train_op 加 表示训练
            feed_dict={
                x: batch_data, 
                y: batch_labels})
        if (i+1) % 500 == 0:
            print('[Train] Step : %d, loss: %4.5f, acc: %4.5f'
                 % (i+1, loss_val, acc_val))
        if (i+1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data,
                        y: test_batch_labels
                    })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test ] Step: %d, acc: %4.5f' % (i+1, test_acc))

[Train] Step : 500, loss: 1.61201, acc: 0.25000
[Train] Step : 1000, loss: 1.62513, acc: 0.35000
[Train] Step : 1500, loss: 1.43248, acc: 0.50000
[Train] Step : 2000, loss: 1.50128, acc: 0.50000
[Train] Step : 2500, loss: 1.64675, acc: 0.35000
[Train] Step : 3000, loss: 1.22293, acc: 0.65000
[Train] Step : 3500, loss: 1.42951, acc: 0.50000
[Train] Step : 4000, loss: 1.10305, acc: 0.55000
[Train] Step : 4500, loss: 1.59322, acc: 0.40000
[Train] Step : 5000, loss: 1.40828, acc: 0.40000
(10000, 3072)
(10000,)
[Test ] Step: 5000, acc: 0.47850
[Train] Step : 5500, loss: 1.48131, acc: 0.50000
[Train] Step : 6000, loss: 1.28052, acc: 0.45000
[Train] Step : 6500, loss: 1.18506, acc: 0.65000
[Train] Step : 7000, loss: 1.45286, acc: 0.50000
[Train] Step : 7500, loss: 1.58170, acc: 0.40000
[Train] Step : 8000, loss: 1.34222, acc: 0.60000
[Train] Step : 8500, loss: 1.30671, acc: 0.60000
[Train] Step : 9000, loss: 1.02233, acc: 0.60000
[Train] Step : 9500, loss: 1.34093, acc: 0.55000
[Train] Step :

AssertionError: Nesting violated for default stack of <class 'tensorflow.python.client.session.Session'> objects